In [202]:
from IPython.core.display import display, HTML
from bs4 import BeautifulSoup
import urllib.request
import re
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import time
from collections import defaultdict

In [203]:
#URL for Jonah Hill's IMDB Page
jonah_url = 'https://www.imdb.com/name/nm1706767/?ref_=nv_sr_srsg_0'

In [204]:
r_jonah = requests.get(jonah_url)
jonah_urlText = r_jonah.text
jonah_soup = BeautifulSoup(jonah_urlText, 'html.parser')

In [205]:
print(jonah_soup.prettify())

<!DOCTYPE html>
<html xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:og="http://ogp.me/ns#">
 <head>
  <meta charset="utf-8"/>
  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
  <meta content="app-id=342792525, app-argument=imdb:///name/nm1706767?src=mdot" name="apple-itunes-app"/>
  <script type="text/javascript">
   var IMDbTimer={starttime: new Date().getTime(),pt:'java'};
  </script>
  <script>
   if (typeof uet == 'function') {
      uet("bb", "LoadTitle", {wb: 1});
    }
  </script>
  <script>
   (function(t){ (t.events = t.events || {})["csm_head_pre_title"] = new Date().getTime(); })(IMDbTimer);
  </script>
  <title>
   Jonah Hill - IMDb
  </title>
  <script>
   (function(t){ (t.events = t.events || {})["csm_head_post_title"] = new Date().getTime(); })(IMDbTimer);
  </script>
  <script>
   if (typeof uet == 'function') {
      uet("be", "LoadTitle", {wb: 1});
    }
  </script>
  <script>
   if (typeof uex == 'function') {
      uex("ld", "LoadTitle", {wb: 1});
    }

In [206]:
#initialize lists for movies, years, and imdb_ids to be added to a dataframe later
movies = []
years = []
imdb_ids = []
#loop through every entry in filmography where Jonah Hill was an actor
for entry in jonah_soup.find(id = 'filmography').find_all('div', id = re.compile('actor.*')):
    #if this is an entry with a movie in it and it is not defined as TV, 
    #add it to the movie list and add the year to the year list
    if entry.find('b') and not ('TV' in entry.text) and not ('Video' in entry.text):
        movies.append(entry.find('b').text)
        imdb_ids.append(entry.get('id')[6:])
    if entry.find('span', class_ = 'year_column') and not ('TV' in entry.text) and not ('Video' in entry.text):
        myString = entry.find('span', class_ = 'year_column').text
        years.append(int(myString[2:6]))
assert(len(imdb_ids) == len(years) == len(movies))

In [207]:
jonah_hill_df = pd.DataFrame()
jonah_hill_df['Movie'] = movies
jonah_hill_df['Year'] = years
jonah_hill_df.index = imdb_ids
jonah_hill_df

,Movie,Year
tt6511932,The Beach Bum,2019
tt3513498,The Lego Movie 2: The Second Part,2019
tt2386490,How to Train Your Dragon: The Hidden World,2019
tt6288124,"Don't Worry, He Won't Get Far on Foot",2018
tt4116284,The Lego Batman Movie,2017
tt2005151,War Dogs,2016
tt1700841,Sausage Party,2016
tt0475290,"Hail, Caesar!",2016
tt2273657,True Story,2015
tt2294449,22 Jump Street,2014


In [208]:
#Section for getting box office information
box_office_url = 'http://boxofficemojo.com'
domestic_money = []
international_money = []
ids_to_delete = []
for mov_id in imdb_ids:
    print(jonah_hill_df['Movie'][mov_id])
    r_box_office = requests.get(box_office_url + '/title/' + mov_id)
    box_office_text = r_box_office.text
    box_office_soup = BeautifulSoup(box_office_text, 'html.parser')
    table = box_office_soup.find('div', {'class':'mojo-performance-summary-table'})
    for entry in table.find_all(class_ = 'a-section a-spacing-none'):
        if entry.find(class_ = 'a-size-small') and entry.find(class_ = 'money'):
            if 'Domestic' in entry.find(class_ = 'a-size-small').text:
                domestic_money.append(int(re.sub('[\$,]','', entry.find(class_ = 'money').text)))
            if 'International' in entry.find(class_ = 'a-size-small').text:
                international_money.append(int(re.sub('[\$,]','', entry.find(class_ = 'money').text)))
        else:
            ids_to_delete.append(mov_id)
    time.sleep(.75)
jonah_hill_df.drop(ids_to_delete,inplace = True)
print(domestic_money)
print(international_money)
assert(len(domestic_money) == len(international_money) == len(jonah_hill_df['Movie']))

The Beach Bum
The Lego Movie 2: The Second Part
How to Train Your Dragon: The Hidden World
Don't Worry, He Won't Get Far on Foot
The Lego Batman Movie
War Dogs
Sausage Party
Hail, Caesar!
True Story
22 Jump Street
How to Train Your Dragon 2
The Lego Movie
The Wolf of Wall Street
This Is the End
Django Unchained
The Watch
21 Jump Street
The Sitter
Moneyball
Megamind
Get Him to the Greek
How to Train Your Dragon
Cyrus
The Invention of Lying
Funny People
Night at the Museum: Battle of the Smithsonian
Just Add Water
Forgetting Sarah Marshall
Horton Hears a Who!
Strange Wilderness
Walk Hard: The Dewey Cox Story
Superbad
Evan Almighty
Knocked Up
Rocket Science
10 Items or Less
Accepted
Click
Grandma's Boy
Pancho's Pizza
The 40-Year-Old Virgin
I Heart Huckabees
[3502600, 105806508, 160799505, 2793039, 175750384, 43034523, 97685686, 30498085, 4719695, 191719337, 177002924, 257760692, 116900694, 101470202, 162805434, 35353000, 138447667, 30441326, 75605492, 148415853, 60974475, 217581231, 74689

In [209]:
jonah_hill_df['Domestic'] = domestic_money
jonah_hill_df['International'] = international_money
jonah_hill_df['Total'] = [domestic_money[x] + international_money[x] for x in range(len(domestic_money))]

In [210]:
jonah_hill_df.index.name = 'IMDB ID'
jonah_hill_df.head()

,Movie,Year,Domestic,International,Total
IMDB ID,,,,,
tt6511932,The Beach Bum,2019,3502600,1050586,4553186
tt3513498,The Lego Movie 2: The Second Part,2019,105806508,85500000,191306508
tt2386490,How to Train Your Dragon: The Hidden World,2019,160799505,359097143,519896648
tt6288124,"Don't Worry, He Won't Get Far on Foot",2018,2793039,2793039,5586078
tt4116284,The Lego Batman Movie,2017,175750384,136200000,311950384


In [214]:
yearly_gross = jonah_hill_df.groupby('Year').sum()[['Domestic', 'International', 'Total']]
yearly_gross.head()

,Domestic,International,Total
Year,,,
2004,12785432,7309477,20094909
2005,109449237,67929408,177378645
2006,179852601,104299206,284151807
2007,389726535,193979304,583705839
2008,224277184,184998679,409275863


In [215]:
yearly_gross.to_csv('yearly_profit.csv')

In [221]:
yearly_gross

,Domestic,International,Total
Year,,,
2004,12785432,7309477,20094909
2005,109449237,67929408,177378645
2006,179852601,104299206,284151807
2007,389726535,193979304,583705839
2008,224277184,184998679,409275863
2009,247550017,269547895,517097912
2010,434440495,483509363,917949858
2011,106046818,39101586,145148404
2012,336606101,358615327,695221428


In [230]:
three_year_profits = pd.DataFrame()
three_year_profits['Years'] = ['2004-2006', '2007-2009', '2010-2012', '2013-2015', '2016-2018']
three_year_profits['Total'] = [
    sum(yearly_gross['Total'][0:3]), 
    sum(yearly_gross['Total'][3:6]), 
    sum(yearly_gross['Total'][6:9]), 
    sum(yearly_gross['Total'][10:13]), 
    sum(yearly_gross['Total'][13:16])]
three_year_profits

,Years,Total
0,2004-2006,481625361
1,2007-2009,1510079614
2,2010-2012,1758319690
3,2013-2015,1716781183
4,2016-2018,1033292804


In [231]:
three_year_profits.set_index('Years', drop = True, inplace = True)

In [232]:
three_year_profits.to_csv('threeYearProfits.csv')

In [236]:
movie_and_domestic_df = jonah_hill_df.set_index('Movie', drop = True)[['Domestic']]
movie_and_domestic_df.to_csv('movie_and_domestic.csv')

In [237]:
movie_and_international_df = jonah_hill_df.set_index('Movie', drop = True)[['International']]
movie_and_international_df.to_csv('movie_and_international.csv')